In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as pl
from matplotlib import animation
from IPython.display import HTML
figsize_s = 6,4
figsize_l = 9,4
pl.rcParams.update({'legend.fontsize' : 'medium'})

In [ ]:
Omega = 7.29e-5 # Angular velocity of the earth.
g = 9.81        # Gravity constant.
a = 6.371e6     # Earth radius.

In [ ]:
f_multiplier = 1.
lat = 45.
f0 = f_multiplier * 2.*Omega*np.sin(np.pi/180.*lat)
L = a * (2.*np.pi) #* np.cos(np.pi/180.*lat)
L_dist = 1.e6
h_bar = 100.
h_prime = 0.1

In [ ]:
# Here we initialize the grid and the horizontal velocities.
nx = 2560
x = np.linspace(-L/2., L/2., nx)
dx = x[1]-x[0]
u0 = np.zeros(nx)
v0 = np.zeros(nx)

# The perturbation in h has the shape of a Gaussian (normal distribution).
# x_sigma = L_dist/6.
# h0 = np.exp(-x**2/(2.*x_sigma**2))
# h0 = - h_prime*h0/np.max(h0)

# Mexican hat (Ricker wavelet).
h_sigma = 25.*dx # The wavelet is approximately 8 h_sigma in total width.
h_width = 2./((3.*h_sigma)**.5*np.pi**.25)
h0 = 2./((3.*h_sigma)**.5*np.pi**.25) * (1. - x**2/h_sigma**2) * np.exp(-x**2/(2.*h_sigma**2))
h0 = h_prime*h0/np.max(h0)

In [ ]:
nt = 80000
ni = 500
dt = 10.

In [ ]:
# MODEL INTEGRATOR, THIS MIGHT TAKE A WHILE!
u = np.copy(u0)
v = np.copy(v0)
h = np.copy(h0)

h_out = np.zeros((nt//ni+1, nx))
h_out[0,:] = h0

for n in range(1,nt+1):
    u += dt * (   f0*v - g*np.gradient(h,dx) )
    v += dt * ( - f0*u )
    h += dt * ( - h_bar*np.gradient(u,dx) )
    
    if (n%ni == 0):
        h_out[n//ni,:] = h[:]

In [ ]:
# ANIMATION CREATION PHASE 1. PLEASE IGNORE THIS FIGURE, WE DO NOT KNOW HOW TO SUPPRESS.
# Set up the figure, the axis, and the plot element we want to animate
fig, ax = pl.subplots(figsize=figsize_l)
xmin, xmax = min(x), max(x)
hmin, hmax = np.min(h_out), np.max(h_out)
hmin -= 0.05*(hmax-hmin)
hmax += 0.05*(hmax-hmin)
ax.set_xlim(xmin, xmax)
ax.set_ylim(hmin, hmax)
ax.set_xlabel('x (m)')
ax.set_ylabel('h (m)')
ax.plot(x, h_out[0,:], 'b:')
line, = ax.plot([], [], lw=2)
fig.tight_layout()
pl.close()

In [ ]:
# ANIMATION CREATION PHASE 2
# Initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    line.set_data(x, h_out[i,:])
    return (line,)

# Call the animator. Keyword blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=h_out.shape[0], interval=40, blit=True)

HTML(anim.to_html5_video())

In [ ]:
anim.save('f0_10.mp4', bitrate=-1)